<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%ED%83%80%EC%9E%84%EB%9E%A9%EC%8A%A4_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine을 활용한 타임랩스 생성

In [ ]:
# !pip install -U geemap

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

Pan and zoom to an area of interest and draw a rectangle on the map.

In [ ]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [ ]:
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif="sentinel1.gif",
    start_year=2019,
    end_year=2019,
    start_date="04-01",
    end_date="08-01",
    frequency="day",
    vis_params={"min": -30, "max": 0},
    palette="Greys",
    frames_per_second=3,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)

In [ ]:
geemap.show_image(timelapse)

![](https://i.imgur.com/FlhvI46.gif)